In [4]:

## !pip install git+https://github.com/IBM/tsfm.git

## !pip install transformers




## Source

* https://huggingface.co/blog/patchtst


In [6]:

from transformers import PatchTSTConfig
from tsfm_public.toolkit.dataset import ForecastDFDataset


In [7]:

# Standard
import os

import numpy as np
import pandas as pd


In [8]:



# Third Party
from transformers import (
    EarlyStoppingCallback,
    PatchTSTConfig,
    PatchTSTForPrediction,
    Trainer,
    TrainingArguments,
)



# First Party
from tsfm_public.toolkit.dataset import ForecastDFDataset
from tsfm_public.toolkit.time_series_preprocessor import TimeSeriesPreprocessor
from tsfm_public.toolkit.util import select_by_index



In [9]:

from transformers import set_seed

set_seed(2023)



In [13]:

# The ECL data is available from https://github.com/zhouhaoyi/Informer2020?tab=readme-ov-file#data

dataset_path = "data/ETTh1.csv"

timestamp_column = "date"
id_columns = []

context_length   = 512
forecast_horizon = 96
patch_length     = 16
num_workers      = 16  # Reduce this if you have low number of CPU cores
batch_size       = 64  # Adjust according to GPU memory



In [14]:


data = pd.read_csv(
    dataset_path,
    parse_dates=[timestamp_column],
)


In [15]:

data


,date,HUFL,HULL,MUFL,MULL,LUFL,LULL,OT
0,2016-07-01 00:00:00,5.827,2.009,1.599,0.462,4.203,1.340,30.531000
1,2016-07-01 01:00:00,5.693,2.076,1.492,0.426,4.142,1.371,27.787001
2,2016-07-01 02:00:00,5.157,1.741,1.279,0.355,3.777,1.218,27.787001
3,2016-07-01 03:00:00,5.090,1.942,1.279,0.391,3.807,1.279,25.044001
4,2016-07-01 04:00:00,5.358,1.942,1.492,0.462,3.868,1.279,21.948000
...,...,...,...,...,...,...,...,...
17415,2018-06-26 15:00:00,-1.674,3.550,-5.615,2.132,3.472,1.523,10.904000
17416,2018-06-26 16:00:00,-5.492,4.287,-9.132,2.274,3.533,1.675,11.044000
17417,2018-06-26 17:00:00,2.813,3.818,-0.817,2.097,3.716,1.523,10.271000
17418,2018-06-26 18:00:00,9.243,3.818,5.472,2.097,3.655,1.432,9.778000


In [16]:

forecast_columns = list(data.columns[1:])
forecast_columns


['HUFL', 'HULL', 'MUFL', 'MULL', 'LUFL', 'LULL', 'OT']

In [17]:

# get split
num_train = int(len(data) * 0.7)
num_test = int(len(data) * 0.2)
num_valid = len(data) - num_train - num_test


In [19]:


border1s = [
    0,
    num_train - context_length,
    len(data) - num_test - context_length,
]


In [20]:

border1s


[0, 11682, 13424]

In [21]:

border2s = [num_train, num_train + num_valid, len(data)]


In [22]:

border2s


[12194, 13936, 17420]

In [24]:


train_start_index = border1s[0]  # None indicates beginning of dataset
train_end_index   = border2s[0]

# we shift the start of the evaluation period back by context length so that
# the first evaluation timestamp is immediately following the training data
valid_start_index = border1s[1]
valid_end_index = border2s[1]

test_start_index = border1s[2]
test_end_index   = border2s[2]

train_data = select_by_index(
    data,
    id_columns=id_columns,
    start_index=train_start_index,
    end_index=train_end_index,
)

valid_data = select_by_index(
    data,
    id_columns=id_columns,
    start_index=valid_start_index,
    end_index=valid_end_index,
)

test_data = select_by_index(
    data,
    id_columns=id_columns,
    start_index=test_start_index,
    end_index=test_end_index,
)



In [29]:

time_series_preprocessor = TimeSeriesPreprocessor(
    timestamp_column=timestamp_column,
    id_columns=id_columns,
    input_columns=forecast_columns,
    output_columns=forecast_columns,
    scaling=True,
)

time_series_preprocessor = time_series_preprocessor.train(train_data)



In [38]:


vars( ForecastDFDataset )


mappingproxy({'__module__': 'tsfm_public.toolkit.dataset',
              '__doc__': '\n    A :class: `ForecastDFDataset` used for forecasting.\n\n    Args:\n        data_df (DataFrame, required): input data\n        datetime_col (str, optional): datetime column in the data_df. Defaults to None\n        x_cols (list, optional): list of columns of X. If x_cols is an empty list, all the columns in the data_df is taken, except the datatime_col. Defaults to an empty list.\n        group_ids (list, optional): list of group_ids to split the data_df to different groups. If group_ids is defined, it will triggle the groupby method in DataFrame. If empty, entire data frame is treated as one group.\n        seq_len (int, required): the sequence length. Defaults to 1\n        num_workers (int, optional): the number if workers used for creating a list of dataset from group_ids. Defaults to 1.\n        pred_len (int, required): forecasting horizon. Defaults to 0.\n    ',
              '__init__': <fu

In [ ]:

'''

mappingproxy({
'__module__': 'tsfm_public.toolkit.dataset',
'__doc__': '\n    A :class: `ForecastDFDataset` used for forecasting.\n\n    
Args:\n        
data_df (DataFrame, required): 
input data\n        
datetime_col (str, optional): 
datetime column in the data_df. Defaults to None\n   

x_cols (list, optional): list of columns of X. 

If x_cols is an empty list, all the columns in the data_df is taken, except the datatime_col. 
Defaults to an empty list.\n    

group_ids (list, optional): list of group_ids to split the data_df to different groups. 
If group_ids is defined, it will triggle the groupby method in DataFrame. 
If empty, entire data frame is treated as one group.\n        
seq_len (int, required): the sequence length. Defaults to 1\n        
num_workers (int, optional): the number if workers used for creating a list of dataset from group_ids. 
Defaults to 1.\n        

pred_len (int, required): forecasting horizon. 
Defaults to 0.\n    ',

########################
'__init__': <function tsfm_public.toolkit.dataset.ForecastDFDataset.__init__(self, 
data                       : pandas.core.frame.DataFrame, 
id_columns                 : List[str] = [], 
timestamp_column           : Optional[str] = None, 

target_columns             : List[str] = [], 
observable_columns         : List[str] = [], 
control_columns            : List[str] = [], 
conditional_columns        : List[str] = [], 
static_categorical_columns : List[str] = [], 

context_length             : int = 1, 
prediction_length          : int = 1, 
num_workers: int = 1, 
frequency_token: Optional[int] = None, 
autoregressive_modeling: bool = True)>,
'BaseForecastDFDataset': tsfm_public.toolkit.dataset.ForecastDFDataset.BaseForecastDFDataset,
'__parameters__': ()})

'''


In [39]:

train_dataset = ForecastDFDataset(
    time_series_preprocessor.preprocess(train_data),
    id_columns=id_columns,
    timestamp_column="date",
    
    observable_columns =forecast_columns,
    target_columns=forecast_columns,
    
    context_length=context_length,
    prediction_length=forecast_horizon,
)


In [40]:

valid_dataset = ForecastDFDataset(
    time_series_preprocessor.preprocess(valid_data),
    id_columns=id_columns,
    timestamp_column="date",
    observable_columns=forecast_columns,
    target_columns=forecast_columns,
    context_length=context_length,
    prediction_length=forecast_horizon,
)


In [41]:


test_dataset = ForecastDFDataset(
    time_series_preprocessor.preprocess(test_data),
    id_columns=id_columns,
    timestamp_column="date",
    observable_columns=forecast_columns,
    target_columns=forecast_columns,
    context_length=context_length,
    prediction_length=forecast_horizon,
)


In [42]:

config = PatchTSTConfig(
    num_input_channels=len(forecast_columns),
    context_length=context_length,
    patch_length=patch_length,
    patch_stride=patch_length,
    prediction_length=forecast_horizon,
    random_mask_ratio=0.4,
    d_model=128,
    num_attention_heads=16,
    num_hidden_layers=3,
    ffn_dim=256,
    dropout=0.2,
    head_dropout=0.2,
    pooling_type=None,
    channel_attention=False,
    scaling="std",
    loss="mse",
    pre_norm=True,
    norm_type="batchnorm",
)
model = PatchTSTForPrediction(config)


In [43]:

training_args = TrainingArguments(
    output_dir="./checkpoint/patchtst/electricity/pretrain/output/",
    overwrite_output_dir=True,
    # learning_rate=0.001,
    num_train_epochs=100,
    do_eval=True,
    evaluation_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    dataloader_num_workers=num_workers,
    save_strategy="epoch",
    logging_strategy="epoch",
    save_total_limit=3,
    logging_dir="./checkpoint/patchtst/electricity/pretrain/logs/",  # Make sure to specify a logging directory
    load_best_model_at_end=True,  # Load the best model when training ends
    metric_for_best_model="eval_loss",  # Metric to monitor for early stopping
    greater_is_better=False,  # For loss
    label_names=["future_values"],
)



In [44]:

# Create the early stopping callback
early_stopping_callback = EarlyStoppingCallback(
    early_stopping_patience=10,  # Number of epochs with no improvement after which to stop
    early_stopping_threshold=0.0001,  # Minimum improvement required to consider as improvement
)


In [45]:

# define trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    callbacks=[early_stopping_callback],
    # compute_metrics=compute_metrics,
)

# pretrain
trainer.train()


/Users/user/opt/anaconda3/envs/py39_HF_TimeSeries/lib/python3.9/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
/Users/user/opt/anaconda3/envs/py39_HF_TimeSeries/lib/python3.9/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker nu

Epoch,Training Loss,Validation Loss
1,0.464000,0.321229
2,0.374300,0.313739


/Users/user/opt/anaconda3/envs/py39_HF_TimeSeries/lib/python3.9/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/Users/user/opt/anaconda3/envs/py39_HF_TimeSeries/lib/python3.9/site-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8 (`cpuset` is not taken into account), which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower th

KeyboardInterrupt: 